### Ortschaften der Schweiz
Das File `PLZO_CSV_LV95.csv` enthält eine Tabelle mit Angaben zu Ortschaften in der Schweiz.  
https://en.wikibooks.org/wiki/Algorithm_Implementation/Geometry/Convex_hull/Monotone_chain
https://www.swisstopo.admin.ch/de/schweizer-koordinatensystem
https://www.bfs.admin.ch/bfs/de/home/grundlagen/agvch/identifikatoren-gemeinde.html

Plot Ortschaften
Needs Background image of Switzerland, 
need a Reference Frame if we zoom in.

In [1]:
def to_int_or_str(s):
    if s.replace('.', '', 1).isdigit():
        s = int(float(s))
    return s


# Zeilen des Files in Liste lines lesen, newlines entfernen
SEP = ';'
filename = '../L07/PLZO_CSV_LV95.txt'
with open(filename, mode='r') as f:
    lines = [line.rstrip() for line in f]

colname_idx = {
    'ORT': 0,
    'PLZ': 1,
    'ZZ': 2,
    'GDE': 3,
    'BFS': 4,
    'KT': 5,
    'E': 6,
    'N': 7,
    'LG': 8,
}

table = []
for line in lines[1:]:
    row = line.split(SEP)
    if len(row) != 9:
        print(line)
        break
    row = [to_int_or_str(row[i]) for i in range(9)]
    table.append(row)

i, j = colname_idx['E'], colname_idx['N']
EN_pts = [(row[i], row[j]) for row in table]

In [2]:
EN_pts = set((row[i], row[j]) for row in table)

In [3]:
ll_data = min(pt[0] for pt in EN_pts), min(pt[1] for pt in EN_pts)
ur_data = max(pt[0] for pt in EN_pts), max(pt[1] for pt in EN_pts)
ll_data, ur_data

((2486639, 1076212), (2830503, 1294140))

In [4]:
from ipycanvas import Canvas, hold_canvas
from canvastools import make_transform
from computational_geometry import convex_hull


WIDTH = 350
HEIGHT = 225
MARGIN = 5
layout = {'border': '1px solid black'}
canvas = Canvas(width=WIDTH, height=HEIGHT, layout=layout)
data2cv = make_transform(canvas,
                         ll_data,
                         ur_data,
                         margin=MARGIN,)

canvas.stroke_style = 'blue'
canvas

Canvas(height=225, layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right=…

In [5]:
canvas.clear()

In [6]:
with hold_canvas():
    for pt in EN_pts:
        canvas.fill_circle(*data2cv(*pt), 1)

In [9]:
ch = [data2cv(E, N) for E, N in convex_hull(EN_pts)]
canvas.stroke_polygon(ch)

In [7]:
E, N = 2600000, 1200000
x, y = data2cv(E, N)
x, y

(113.735502989554, 95.03505745016705)

In [8]:
data2cv(x, y, inv=True)

(2600000.0, 1200000.0)

In [33]:
state = {'down': None,
         'up': None,
         'll_data': ll_data,
         'ur_data': ur_data,
         'data2cv': data2cv,
         }


def clip(pts, ll, ur):
    '''ll and ur are just opposite corners of the bounding rectangle'''
    ll, ur = get_ll_ur(ll, ur)
    return [(x, y) for x, y in pts
            if ll[0] <= x <= ur[0] and ll[1] <= y <= ur[1]]


def get_ll_ur(pt1, pt2):
    ll = min(pt1[0], pt2[0]), min(pt1[1], pt2[1])
    ur = max(pt1[0], pt2[0]), max(pt1[1], pt2[1])
    return ll, ur


def redraw():
    ll = state['ll_data']
    ur = state['ur_data']
    data2cv = state['data2cv']
    canvas.clear()

    with hold_canvas():
        for pt in clip(EN_pts, ll, ur):
            canvas.fill_circle(*data2cv(*pt), radius=1)


def on_mouse_down(x, y):
    state['up'] = None
    state['down'] = (x, y)


def on_mouse_up(x, y):
    state['up'] = (x, y)
    ll = state['down']
    ur = state['up']
    state['down'] = None


    if ll and ur:
        state['ll_data'] = state['data2cv'](*ll, inv=True)
        state['ur_data'] = state['data2cv'](*ur, inv=True)
        state['data2cv'] = make_transform(canvas,
                                          state['ll_data'],
                                          state['ur_data'],
                                          margin=MARGIN)
    redraw()

In [39]:
state['down'] = (150, 50)
up = (200, 100)

In [40]:
state

{'down': (150, 50),
 'up': (200, 100),
 'll_data': (2636145.0869565215, 1244610.9090909092),
 'ur_data': (2685980.4492753623, 1195081.8181818181),
 'data2cv': <function canvastools.make_transform.<locals>.xy_data2xy_canvas(x, y, inv=False)>}

In [41]:
canvas

Canvas(height=225, layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right=…

In [42]:
on_mouse_up(*up)

In [29]:
state

{'down': None,
 'up': (200, 100),
 'll_data': (2636145.0869565215, 1244610.9090909092),
 'ur_data': (2685980.4492753623, 1195081.8181818181),
 'data2cv': <function canvastools.make_transform.<locals>.xy_data2xy_canvas(x, y, inv=False)>}

In [30]:
clip(EN_pts, state['ll_data'], state['ur_data'])

[]

In [31]:
len(EN_pts)

5741